In [ ]:
# !/usr/bin/env python
# coding: utf-8

import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from EDA_functions import *
seed = 5000

# 0. CARGA DE DATOS.

In [ ]:
weatherAUS = pd.read_csv('../data/weatherAUS.csv') #

__NOTA:__ Se hará uso de algunas funciones contenidas en el script __EDA_functions.py__.

# 1. TAMAÑO DE DATA - PRIMEROS VISTAZOS GENERALES.

A partir de aquí, se prepará la data de tal forma que pueda soportar un problema de clasificación binaria, es decir, para entrenar un modelo supervisado. Siendo así, las variables de localización y meteorológicas se considerarán como __features__ o __predictoras__. La __variable objetivo__ o __respuesta__ será `RainTomorrow`.



Veamos cuál es el tamaño de data.

In [ ]:
weatherAUS.shape

In [ ]:
weatherAUS.head(2)

In [ ]:
weatherAUS.tail(2)

Veamos qué variables poseen valores perdidos y los porcentajes de missing.

In [ ]:
count_missing = get_count_missing(data=weatherAUS, feature_names=list(weatherAUS.columns))
count_missing

Debido a la presencia de missing en la variable objetivo `RainTomorrow`, eliminamos las filas que no cuenten con información de esta variable.

In [ ]:
wA_NOT_MISS_IN_OBJECTIVE = weatherAUS.dropna(subset=['RainTomorrow'])
wA_NOT_MISS_IN_OBJECTIVE.shape

Debido a que hay algunas variables con un porcentaje importante de valores perdidos, establecemos un umbral de filtrado de variables. Las variables arriba del 11 % de valores perdidos, no serán consideradas.

Siendo así, se procede a eliminar aquellas variables.

In [ ]:
high_missing_names = [ 'Sunshine', 'Evaporation', 'Cloud3pm', 'Cloud9am']
wA_NOT_HIGH_MISSING = wA_NOT_MISS_IN_OBJECTIVE.drop(high_missing_names, axis=1)

Se exhiben los nuevos porcentajes de missing.

In [ ]:
get_count_missing(data=wA_NOT_HIGH_MISSING,
                  feature_names=list(set(weatherAUS.columns)-set(high_missing_names)))

A continuación, se exhiben los tipos de variables que conforman el dataset para conocer su naturaleza.

In [ ]:
wA_NOT_HIGH_MISSING.info()

El dataset está conformado por variables de tipo numérico y categórico. Sin embargo, para las fechas, transformemos la variable `Date`a variable tipo `datetime`.

In [ ]:
wA_NOT_HIGH_MISSING['Date'] = pd.to_datetime(wA_NOT_HIGH_MISSING['Date'])

In [ ]:
wA_NOT_HIGH_MISSING['Date'].head(5)

# 2. TRATAMIENTO DE VARIABLES NUMÉRICAS.

## 2.1 Visualización y Control de outliers.

Se identifican las variables numéricas.

In [ ]:
num_names = list(wA_NOT_HIGH_MISSING.select_dtypes('float64').columns)
num_names

Al contar con un número no tan grande de variables numéricas, es posible una exploración individual de la distribuciones usando histogramas y box-plots.

In [ ]:
for name in num_names:
    print(f'################################## VARIABLE: {name} ################################################')
    fig = px.histogram(wA_NOT_HIGH_MISSING, x=name, nbins=50, marginal='box')
    fig.show()

Podemos observar lo siguiente:
* Algunas variables tiene cierto grado de concentración en el centro y otras están sesgadas.
* Variables sesgadas como `Rainfall` y  `WindSpeed9am` cuentan con claros outliers, es decir, puntos que se alejan no solo de la concentración de la información, sino de la cola natural de la distribución. Estos valores son extremos (valores positivos muy grandes).
* Algunas variables tienen valores negativos, la mayoría de ellas se refieren a la temperatura. Debido a que no se tiene mayor contexto de ellas, no se tomará ninguna acción al respecto.

Para ejercer cierto control de outliers en variables sesgadas a la izquierda, se impondrán cotas superiores a estas variables, más precisamente a `Rainfall` y  `WindSpeed9am`.

In [ ]:
wA_SKEW_UPPER_OUTLIERS = wA_NOT_HIGH_MISSING[(wA_NOT_HIGH_MISSING['Rainfall'] <= 300) | (wA_NOT_HIGH_MISSING['Rainfall'].isna())]
wA_SKEW_UPPER_OUTLIERS = wA_SKEW_UPPER_OUTLIERS[(wA_SKEW_UPPER_OUTLIERS['WindSpeed9am'] <= 100) | (wA_SKEW_UPPER_OUTLIERS['WindSpeed9am'].isna())]

Se compara las distribuciones de las variables implicadas antes y después de outliers.

In [ ]:
skew_num_names = ['Rainfall', 'WindSpeed9am']
for name in skew_num_names:
    print(f'################################## VARIABLE: {name} ################################################')
    print(f'Antes de outliers.')
    fig = px.histogram(wA_NOT_HIGH_MISSING, x=name, nbins=50, marginal='box')
    fig.show()
    print(f'Después de outliers.')
    fig2 = px.histogram(wA_SKEW_UPPER_OUTLIERS, x=name, nbins=50, marginal='box' )
    fig2.show()

Se comparan los tamaños de data antes y después de outliers. Se puede ver que el porcentaje de data eliminada es ínfimo. Si esto fuera lo contrario, es decir, si data eliminada fuera considerble, se tendría que pensar en otras estrategias para control de outliers y sesgos extremos, como por ejemplo las tranformaciones __yeo-johnson__ o __box-cox__.

In [ ]:
print('Porcentaje remanente de la data después de eliminar outliers:', (wA_SKEW_UPPER_OUTLIERS.shape[0]/wA_NOT_HIGH_MISSING.shape[0])*100, '%')

## 2.2 Tratamiento de la correlación.

Analicemos la correlación entre  las variables numéricas. Se utilizará la correlación no paramétrica de Spearman.

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(wA_SKEW_UPPER_OUTLIERS[num_names].corr(method='spearman').abs(), square=True, annot=True, cmap='RdBu',  vmin=-1, vmax=1)
plt.show()

Existen correlaciones altas entre parejas de variables, por ejemplo:
* `MaxTemp`-`Temp3pm`, 0.98
* `Pressure9am`-`Pressure3pm`, 0.96
* `MinTemp`-`Temp9am`, 0.9
* `MaxTemp`-`Temp9am`, 0.89
* `Temp9pm`-`Temp3pm`, 0.86
* `MinTemp`-`MaxTemp`, 0.74
* `MinTemp`-`Temp3pm`, 0.71

Se visualizan las correlaciones de las primeras dos parejas, y las de las últimas dos parejas.

In [ ]:
fig = px.scatter(wA_SKEW_UPPER_OUTLIERS, x='MaxTemp', y='Temp3pm', color="RainTomorrow" )
fig.show()

In [ ]:
# fig = px.scatter(wA_SKEW_UPPER_OUTLIERS, x='Pressure9am', y='Pressure3pm', color="RainTomorrow" )
# fig.show()

In [ ]:
# fig = px.scatter(wA_SKEW_UPPER_OUTLIERS, x='MinTemp', y='MaxTemp', color="RainTomorrow" )
# fig.show()

In [ ]:
# fig = px.scatter(wA_SKEW_UPPER_OUTLIERS, x='MinTemp', y='Temp3pm', color="RainTomorrow" )
# fig.show()

Se toma la decisión de eliminar algunas variables que tengan correlación con alguna otra, esto arriba del 0.7. Lo anterior se hace para evitar la redundancia en la información.

In [ ]:
drop_names_by_corr = get_features_names_drop_by_corr(data=wA_SKEW_UPPER_OUTLIERS,
                                                     list_feature_names=num_names,
                                                     threshold=0.7)

In [ ]:
wA_CORR = wA_SKEW_UPPER_OUTLIERS.drop(drop_names_by_corr, axis=1)

Reindexemos la data.

In [ ]:
wA_CORR.reset_index(drop=True, inplace=True)

Se guarda la primera versión de la data preprocesada.

In [ ]:
pickle.dump(wA_CORR, open('../outputs/wA_CORR.sav', 'wb'))